In [1]:
import pandas as pd
import geopandas as gpd

# Zonas Medellin

In [2]:
zo = ['aguacatala_HL', 'moravia_LH', 'floresta_HH']
# z = zo[0]
for z in zo:
    print(f"Processing zone: {z}")

    zone_edges = gpd.read_file(f'../data/output/shape/project_network_initial/{z}/{z}_edges_proj_net_initial.shp')
    zone_nodes = gpd.read_file(f'../data/output/shape/project_network_initial/{z}/{z}_nodes_proj_net_initial.shp')
    sidewalks = gpd.read_file('../data/input/shape/sidewalks/sidewalks.shp')
    zone_edges = zone_edges.to_crs('epsg:32618')
    sidewalks = sidewalks.to_crs('epsg:32618')

    # Buffer the original geometry to avoid float arithmetic problems (in intersects or within)
    sidewalks_buffer = sidewalks.copy()
    sidewalks_buffer.geometry = sidewalks_buffer.geometry.buffer(0.0001)
    # Manzanas del territorio
    zone_blocks = gpd.read_file(f'../data/output/shape/zone_blocks/{z}/{z}_blocks.shp')
    zone_blocks = zone_blocks.to_crs('epsg:32618')
    # Generar el poligono ampliado de la zona de analisis con un buffer
    poligono_ampliado = zone_blocks.union_all().convex_hull.buffer(500)
    poligono_ampliado_gdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries([poligono_ampliado]), crs=zone_blocks.crs)
    # Seleccionar las manzanas de todo el territorio que caen dentro del poligono ampliado
    sidewalks = sidewalks[sidewalks.geometry.intersects(poligono_ampliado)]
    # Unir los andenes y las calles
    lineas = pd.concat([sidewalks,zone_edges], ignore_index=True)
    # Use union_all to split all the self-intersected LineStrings
    un = lineas.geometry.union_all()
    geometrias = [i for i in un.geoms]
    unary = gpd.GeoDataFrame({"geometry":geometrias}, crs=lineas.crs)
    # Use a spatial join (with within or intersect) to join the two dataframes and retrieve the original attributes
    sidewalks_splits = gpd.sjoin(unary, sidewalks_buffer, how="left", predicate='within')
    sidewalks_splits = sidewalks_splits.dropna()
    sidewalks_splits = sidewalks_splits.drop(['index_right'], axis=1)
    sidewalks_splits['sidewalk'] = sidewalks_splits['sidewalk'].astype(int)

    # Create a buffer around the street segments (e.g., 20 meters)
    zone_edges['buffer'] = zone_edges['geometry'].buffer(20)  # Adjust buffer size as needed

    # Create a column to store the proportion of sidewalk presence
    zone_edges['sidewalk'] = 0.0

    # Iterate over each buffer and calculate the lengths of sidewalks within it
    for idx, edge in zone_edges.iterrows():

        # Filter the sidewalks contained in the buffer
        sidewalks_in_buffer = sidewalks_splits[sidewalks_splits['geometry'].within(edge['buffer'])]

        # Calculate the total length of sidewalks within the buffer
        total_length = sidewalks_in_buffer['geometry'].length.sum()
        
        # Calculate the length of sidewalks with sidewalk=1
        length_with_sidewalks = sidewalks_in_buffer[sidewalks_in_buffer['sidewalk'] == 1]['geometry'].length.sum()
        
        # Avoid division by zero
        if total_length > 0:
            # Calculate the proportion of sidewalks (length of sidewalks with sidewalk=1 / total length)
            zone_edges.at[idx, 'sidewalk'] = length_with_sidewalks / total_length

    zone_edges = zone_edges.drop(['buffer'], axis=1)
    zone_edges = zone_edges.to_crs('epsg:4326')
    zone_nodes = zone_nodes.to_crs('epsg:4326')
    zone_edges.to_file(f'../data/output/shape/physical_variables/sidewalks/{z}/{z}_sidewalks_edges.shp')
    zone_nodes.to_file(f'../data/output/shape/physical_variables/sidewalks/{z}/{z}_sidewalks_nodes.shp')

    # Limit the study area to the neighborhood polygon
    edges = zone_edges.copy()
    nodes = zone_nodes.copy()

    edges = edges.reset_index()
    edges = edges.to_crs('epsg:32618')
    nodes = nodes.reset_index()
    nodes = nodes.to_crs('epsg:32618')

    if z == 'floresta_HH':
        w = 'Alto_Alta'
    elif z == 'moravia_LH':
        w = 'Bajo_Alta'
    elif z == 'aguacatala_HL':
        w = 'Alto_Baja'

    # Load polygon files
    polygon = gpd.read_file(f'../data/input/shape/Poligonos/{w}.shp')
    polygon = polygon.to_crs('epsg:32618')
    polygon = polygon['geometry'].union_all()
    polygon = gpd.GeoDataFrame(geometry = [polygon], crs = 'epsg:32618')

    # Select the segments that are inside the polygon
    for idx, row in polygon.iterrows():
        # Filter the edges contained in the polygon
        edges_in_polygon = edges[edges['geometry'].intersects(row['geometry'])]

    # List of the nodes inside the polygon based on the 'u' and 'v' columns of the edges inside the polygon
    nodes_in_polygon = edges_in_polygon['u'].to_list() + edges_in_polygon['v'].to_list()
    # Make the nodes unique and sort it
    nodes_in_polygon = list(set(nodes_in_polygon))
    nodes_in_polygon.sort()
    # Select the nodes inside the buffer
    nodes_in_polygon = nodes.set_index('osmid').loc[nodes_in_polygon].reset_index()

    # Export
    nodes_in_polygon = nodes_in_polygon.to_crs('epsg:4326')
    edges_in_polygon = edges_in_polygon.to_crs('epsg:4326')
    nodes_in_polygon.to_file(f'../data/output/shape/project_network_filter/{z}/{z}_sidewalks_nodes.shp')
    edges_in_polygon.to_file(f'../data/output/shape/project_network_filter/{z}/{z}_sidewalks_edges.shp')

Processing zone: aguacatala_HL
Processing zone: moravia_LH
Processing zone: floresta_HH


# Medellin

In [3]:
z = 'medellin'

zone_edges = gpd.read_file(f'../data/output/shape/project_network_initial/{z}/{z}_edges_proj_net_initial.shp')
zone_nodes = gpd.read_file(f'../data/output/shape/project_network_initial/{z}/{z}_nodes_proj_net_initial.shp')
sidewalks = gpd.read_file('../data/input/shape/sidewalks/sidewalks.shp')
zone_edges = zone_edges.to_crs('epsg:32618')
sidewalks = sidewalks.to_crs('epsg:32618')

# Buffer the original geometry to avoid float arithmetic problems (in intersects or within)
sidewalks_buffer = sidewalks.copy()
sidewalks_buffer.geometry = sidewalks_buffer.geometry.buffer(0.0001)
# Manzanas del territorio
zone_blocks = gpd.read_file(f'../data/output/shape/blocks/{z}_blocks.shp')
zone_blocks = zone_blocks.to_crs('epsg:32618')
# Generar el poligono ampliado de la zona de analisis con un buffer
poligono_ampliado = zone_blocks.union_all().convex_hull.buffer(500)
poligono_ampliado_gdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries([poligono_ampliado]), crs=zone_blocks.crs)
# Seleccionar las manzanas de todo el territorio que caen dentro del poligono ampliado
sidewalks = sidewalks[sidewalks.geometry.intersects(poligono_ampliado)]
# Unir los andenes y las calles
lineas = pd.concat([sidewalks,zone_edges], ignore_index=True)
# Use union_all to split all the self-intersected LineStrings
un = lineas.geometry.union_all()
geometrias = [i for i in un.geoms]
unary = gpd.GeoDataFrame({"geometry":geometrias}, crs=lineas.crs)
# Use a spatial join (with within or intersect) to join the two dataframes and retrieve the original attributes
sidewalks_splits = gpd.sjoin(unary, sidewalks_buffer, how="left", predicate='within')
sidewalks_splits = sidewalks_splits.dropna()
sidewalks_splits = sidewalks_splits.drop(['index_right'], axis=1)
sidewalks_splits['sidewalk'] = sidewalks_splits['sidewalk'].astype(int)

# Create a buffer around the street segments (e.g., 20 meters)
zone_edges['buffer'] = zone_edges['geometry'].buffer(20)  # Adjust buffer size as needed

# Create a column to store the proportion of sidewalk presence
zone_edges['sidewalk'] = 0.0

# Iterate over each buffer and calculate the lengths of sidewalks within it
for idx, edge in zone_edges.iterrows():

    # Filter the sidewalks contained in the buffer
    sidewalks_in_buffer = sidewalks_splits[sidewalks_splits['geometry'].within(edge['buffer'])]

    # Calculate the total length of sidewalks within the buffer
    total_length = sidewalks_in_buffer['geometry'].length.sum()
    
    # Calculate the length of sidewalks with sidewalk=1
    length_with_sidewalks = sidewalks_in_buffer[sidewalks_in_buffer['sidewalk'] == 1]['geometry'].length.sum()
    
    # Avoid division by zero
    if total_length > 0:
        # Calculate the proportion of sidewalks (length of sidewalks with sidewalk=1 / total length)
        zone_edges.at[idx, 'sidewalk'] = length_with_sidewalks / total_length

zone_edges = zone_edges.drop(['buffer'], axis=1)
zone_edges = zone_edges.to_crs('epsg:4326')
zone_nodes = zone_nodes.to_crs('epsg:4326')
zone_edges.to_file(f'../data/output/shape/physical_variables/sidewalks/{z}/{z}_sidewalks_edges.shp')
zone_nodes.to_file(f'../data/output/shape/physical_variables/sidewalks/{z}/{z}_sidewalks_nodes.shp')